In [1]:
from konlpy.corpus import kobill

# lexicon을 만드는 함수 정의
def getUniqueTerms():
    lexicon = list() # unique terms list
    # 사전(lexicon) 만들기 -> 우리가 수집한 데이터(crawled data)를 N차원으로 표현
    for fileName in kobill.fileids():
        document = kobill.open(fileName).read()
        for token in document.split():
            if token not in lexicon: # token이 lexicon에 있는지 검사 후 있으면 추가를 하지 않고 없으면 추가를 한다.
                lexicon.append(token)
        return lexicon

def getUniqueTermsBySet():
    lexicon = list() # unique terms list
    # 사전(lexicon) 만들기 -> 우리가 수집한 데이터(crawled data)를 N차원으로 표현
    for fileName in kobill.fileids():
        document = kobill.open(fileName).read()
        for token in document.split():
            lexicon.append(token)
    return list(set(lexicon)) # set을 함으로서 중복된 값들이 사라진다.
print(kobill.fileids())

['1809890.txt', '1809891.txt', '1809892.txt', '1809893.txt', '1809894.txt', '1809895.txt', '1809896.txt', '1809897.txt', '1809898.txt', '1809899.txt']


In [86]:
%timeit getUniqueTerms()

2.81 ms ± 333 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [89]:
%timeit getUniqueTermsBySet()

4.04 ms ± 179 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [91]:
lexicon2 = getUniqueTermsBySet()
# print (len(lexicon2))

In [92]:
from collections import Counter, defaultdict

def docRepresentation(lexicon): #사전에 있는 맞는 단어를 받아 Bow로 표현해줌.
    # 문서를 전체 lexicon이라는 사전으로 표현한 후 사전의 전체 개수(유니크한 단어의 수)를 N개라고 할 때 N차원으로 문서를 하나 표현하고자 함. 
    # 단어가 있으면 1, 없으면 0으로 표현한 후 append. 따라서 사전에 위치한 단어의 인덱스를 받아올 필요가 있으며, 그 결과 1개의 문서에 대한 벡터 표현이 가능해진다. 
    
    
    docList = list()
    for fileName in kobill.fileids():
        document = kobill.open(fileName).read()
        docVector = list(0 for _ in range(len(lexicon))) # lexicon의 사이즈만큼 0부터 돌아가면서 0으로 초기화한다. 즉 2300차원 정도를 갖는 0으로 초기화된 벡터가 됨. 
        
        for token in document.split():
            docVector[lexicon.index(token)] = 1 # 몇번째에 위치하는지 인덱스를 받고 1을 넣어줌.
        docList.append(docVector) # 전체에 대해서 append
    return docList


def docRepresentationByDefaultDict(lexicon): #사전에 있는 맞는 단어를 받아 Bow로 표현해줌.(defaultdict로 속도 개선)  
    docList = defaultdict(lambda: defaultdict(int)) # 람다함수로 초기화시켜주어야 함.
    
    for fileName in kobill.fileids():
        document = kobill.open(fileName).read()
        
        for token in document.split():
            docVector[document][token] = 1 # 몇번째에 위치하는지 인덱스를 받고 1을 넣어줌.
    return docList

# --> bag of word
# Document-Term Matrix라고 하여 DTM으로 부른다. 
#     w1, w2, w3, ..., w2300
# d1   1,  0,  0, ..., => sparse하다
# d2
# d3
# ...
# d10
# dict{dict{}} 구조로 되어 있는 형태. 

In [93]:
%timeit docRepresentation(lexicon)

ValueError: '국가공무원법' is not in list

In [60]:
%timeit docRepresentationByDict(lexicon)

NameError: name 'docVector' is not defined

In [94]:
docList = docRepresentationByDefaultDict(lexicon)

NameError: name 'docVector' is not defined

In [62]:
docList, len(docList)

NameError: name 'docList' is not defined

In [63]:
# boolean 검색 모델 만들기(있거나 없거나 여부만 검색하므로)
# information retrieval에는 불리언과 vector space model 두개로 크게 나뉘어진다.


query = ["국회", "의원"]
result = list()

def retrievalByBoolean(): # microsec
    result = list()
    # time complexity가 너무 높다는 단점이 존재. modle time complecity가 승수인 경우 좋지 않은 성능을 보임. 
    for token in query: # 이터레이션을 두번 돔
        candidate = list()
        for docName, docVector in docList.items(): # items로 key-value쌍을 가져옴.
            for term, freq in docVector:items(): # freq:있는지 없는지 여부
                if term == token:
                    candidate.append(docName)
        result.append(candidate)
    
def retrievalByTDM(): # nanosec
    result = list()
    for token in query:
        result.append(TDM[token])
    return result
    
print(result)
print(list(set(result[0]).intersection(result[1])))

SyntaxError: invalid syntax (<ipython-input-63-f3ff23578ff9>, line 12)

In [ ]:
# 이 구조는 파이썬에서만 사용 가능. Dictionary | Postiong DB로 구성
# 다른 언어에서 짜려면(eg. C) 다음과 같이 코딩 :
    # Dictionary => term, fp (메모리에 Hash 형태로 관리되므로 매우 빠름)
    # Postiong => struct(docid, freq, next=fp) (fileDB)
    
def invertedDocument(DTM): # Transpose를 하지 않는 이유는 우리가 다루고자 하는 데이터가 몇십, 몇백만이기 때문에 Numpy array를 활용할 수 없기 때문. 
    TDM = defaultdict(lambda: defaultdict(int))
    
    for docName, docVector in DTM.items():
        for term, freq in docVector.items():
            TDM[term][docName] = freq
    return TDM

In [49]:
# Inverted document(역문헌구조)를 통해 효율성을 높이고자 함. 

# Document-Term Matrix(DTM) 
#     w1, w2, w3, ..., w2300
# d1   1,  0,  0, ..., => sparse하다
# d2
# d3
# ...
# d10

# Term-Document Matrix(TDM) -> 단순히 축만 바뀐 형태.  
#     d1, d2, d3, ..., w10
# w1   1,  0,  0, ..., => sparse=>dict
# w2
# w3
# ...
# w2300

In [65]:
TDM = invertedDocument(docList)

NameError: name 'invertedDocument' is not defined

In [66]:
TDM["국회"], TDM["의원"]

NameError: name 'TDM' is not defined